In [1]:
# Import Libraries and Load Data
import pandas as pd
import re
import sqlite3

# Load CSV Data
orders_df = pd.read_csv('orders.csv')
print("Orders Data Loaded:", orders_df.shape)

# Load JSON Data
users_df = pd.read_json('users.json')
print("Users Data Loaded:", users_df.shape)

Orders Data Loaded: (10000, 6)
Users Data Loaded: (3000, 4)


In [2]:
# Parse SQL Data
restaurant_data = []

with open('restaurants.sql', 'r') as f:
    sql_content = f.read()
    # Regex pattern to extract values: (id, 'name', 'cuisine', rating)
    pattern = re.compile(r"\(\s*(\d+),\s*'([^']*)',\s*'([^']*)',\s*([0-9.]+)\s*\)")
    matches = pattern.findall(sql_content)
    
    for match in matches:
        restaurant_data.append({
            'restaurant_id': int(match[0]),
            'restaurant_name': match[1],
            'cuisine': match[2],
            'rating': float(match[3])
        })

restaurants_df = pd.DataFrame(restaurant_data)
print("Restaurants Data Loaded:", restaurants_df.shape)

Restaurants Data Loaded: (500, 4)


In [3]:
# Merge Datasets
# Merge orders with users (Left Join)
merged_df = pd.merge(orders_df, users_df, on='user_id', how='left')

# Merge with restaurants (Left Join)
# Note: 'restaurant_name' exists in both orders and restaurants, so we use suffixes
merged_df = pd.merge(merged_df, restaurants_df, on='restaurant_id', how='left', suffixes=('_order', '_details'))

# Save the final dataset
merged_df.to_csv('final_food_delivery_dataset.csv', index=False)
print("Final Dataset Created:", merged_df.shape)
merged_df.head()

Final Dataset Created: (10000, 12)


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_order,name,city,membership,restaurant_name_details,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [4]:
# MCQ 1 - City with Highest Gold Revenue
# Filter for Gold members
gold_members = merged_df[merged_df['membership'] == 'Gold']

# Group by city and sum total_amount
city_revenue = gold_members.groupby('city')['total_amount'].sum().sort_values(ascending=False)
print("City with highest Gold revenue:")
print(city_revenue.head(1))

City with highest Gold revenue:
city
Chennai    1080909.79
Name: total_amount, dtype: float64


In [5]:
# MCQ 2 - Cuisine with Highest Average Order Value
cuisine_avg = merged_df.groupby('cuisine')['total_amount'].mean().sort_values(ascending=False)
print("Cuisine with highest Average Order Value:")
print(cuisine_avg.head(1))

Cuisine with highest Average Order Value:
cuisine
Mexican    808.021344
Name: total_amount, dtype: float64


In [6]:
# MCQ 3 - Users with > ₹1000 Total Spend
user_spend = merged_df.groupby('user_id')['total_amount'].sum()
high_spenders = user_spend[user_spend > 1000].count()
print(f"Users with > 1000 spend: {high_spenders}")

Users with > 1000 spend: 2544


In [7]:
# MCQ 4 - Revenue by Rating Range
def get_rating_range(rating):
    if 3.0 <= rating <= 3.5: return '3.0 – 3.5'
    elif 3.6 <= rating <= 4.0: return '3.6 – 4.0'
    elif 4.1 <= rating <= 4.5: return '4.1 – 4.5'
    elif 4.6 <= rating <= 5.0: return '4.6 – 5.0'
    return 'Other'

merged_df['rating_range'] = merged_df['rating'].apply(get_rating_range)
rating_revenue = merged_df.groupby('rating_range')['total_amount'].sum().sort_values(ascending=False)
print("Highest Revenue Rating Range:")
print(rating_revenue.head(1))

Highest Revenue Rating Range:
rating_range
4.6 – 5.0    2197030.75
Name: total_amount, dtype: float64


In [8]:
# MCQ 5 - Gold Member Average Order Value by City
gold_city_avg = gold_members.groupby('city')['total_amount'].mean().sort_values(ascending=False)
print("Highest Avg Order Value City (Gold):")
print(gold_city_avg.head(1))

Highest Avg Order Value City (Gold):
city
Chennai    808.45908
Name: total_amount, dtype: float64


In [9]:
# MCQ 6 - Cuisine with Lowest Distinct Restaurants
cuisine_stats = merged_df.groupby('cuisine').agg(
    distinct_restaurants=('restaurant_id', 'nunique'),
    total_revenue=('total_amount', 'sum')
).sort_values(by='distinct_restaurants')

print("Cuisine with lowest distinct restaurants:")
print(cuisine_stats.head(1))

Cuisine with lowest distinct restaurants:
         distinct_restaurants  total_revenue
cuisine                                     
Chinese                   120     1930504.65


In [10]:
# MCQ 7 - Percentage of Gold Orders
total_orders = len(merged_df)
gold_orders = len(merged_df[merged_df['membership'] == 'Gold'])
percentage = (gold_orders / total_orders) * 100
print(f"Percentage of Gold orders: {round(percentage)}%")

Percentage of Gold orders: 50%


In [11]:
# MCQ 8 - High Value Restaurant with < 20 Orders
restaurant_stats = merged_df.groupby('restaurant_name_order').agg(
    order_count=('order_id', 'count'),
    avg_order_value=('total_amount', 'mean')
)

# Filter < 20 orders
filtered = restaurant_stats[restaurant_stats['order_count'] < 20]

# Check specific options from the question
options = ["Grand Cafe Punjabi", "Grand Restaurant South Indian", "Ruchi Mess Multicuisine", "Ruchi Foods Chinese"]
print(filtered[filtered.index.isin(options)].sort_values(by='avg_order_value', ascending=False))

                       order_count  avg_order_value
restaurant_name_order                              
Ruchi Foods Chinese             19       686.603158


In [12]:
# MCQ 9 - Highest Revenue Combination
combo_revenue = merged_df.groupby(['membership', 'cuisine'])['total_amount'].sum().sort_values(ascending=False)
print("Top Revenue Combinations:")
print(combo_revenue.head())

Top Revenue Combinations:
membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Name: total_amount, dtype: float64


In [13]:
# MCQ 10 - Quarterly Revenue
# Convert order_date to datetime
merged_df['order_date'] = pd.to_datetime(merged_df['order_date'], dayfirst=True)
merged_df['quarter'] = merged_df['order_date'].dt.quarter

quarter_revenue = merged_df.groupby('quarter')['total_amount'].sum().sort_values(ascending=False)
print("Highest Revenue Quarter:")
print(quarter_revenue.head(1))

Highest Revenue Quarter:
quarter
3    2037385.1
Name: total_amount, dtype: float64


In [14]:
# Numerical Answers
# 1. Total orders by Gold members
print("1. Total Gold Orders:", len(merged_df[merged_df['membership'] == 'Gold']))

# 2. Total revenue from Hyderabad (Rounded)
hyd_revenue = merged_df[merged_df['city'] == 'Hyderabad']['total_amount'].sum()
print("2. Hyderabad Revenue:", round(hyd_revenue))

# 3. Distinct users who placed at least one order
print("3. Distinct Users:", merged_df['user_id'].nunique())

# 4. Average order value for Gold members (2 decimals)
gold_avg = merged_df[merged_df['membership'] == 'Gold']['total_amount'].mean()
print("4. Gold Avg Order Value:", round(gold_avg, 2))

# 5. Orders for restaurants with rating >= 4.5
print("5. Orders with Rating >= 4.5:", len(merged_df[merged_df['rating'] >= 4.5]))

# 6. Orders in top revenue city for Gold members
# We already know Chennai is the top city from above Cell
chennai_gold_orders = len(merged_df[(merged_df['city'] == 'Chennai') & (merged_df['membership'] == 'Gold')])
print("6. Gold Orders in Top City (Chennai):", chennai_gold_orders)

1. Total Gold Orders: 4987
2. Hyderabad Revenue: 1889367
3. Distinct Users: 2883
4. Gold Avg Order Value: 797.15
5. Orders with Rating >= 4.5: 3374
6. Gold Orders in Top City (Chennai): 1337
